In [1]:
import os
import requests
from bs4 import BeautifulSoup
import zipfile
import xport, csv
import pandas as pd
from pandas.api.types import infer_dtype

Define the url to extract from

In [2]:
url = 'https://www.chicagofed.org/banking/financial-institution-reports/commercial-bank-data-complete-2001-2010'
root = 'https://www.chicagofed.org/'

Extract and clean the relevant links

In [3]:
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
all_hrefs = soup.find_all('a')
all_links = [link.get('href') for link in all_hrefs]
zip_files = [dl for dl in all_links if '.zip' in dl or '.ZIP' in dl]
zip_files = [dl.lower() for dl in zip_files]
zip_files = [dl[:dl.index('.zip')+len('.zip')] for dl in zip_files]
download_folder = 'C:/Users/kwang648/Downloads/banking/'

download zip files

In [4]:
if not os.path.exists(download_folder):
    os.makedirs(download_folder)

for zip_file in zip_files:
    full_url = root + zip_file
    r = requests.get(full_url)
    zip_filename = os.path.basename(zip_file)
    dl_path = os.path.join(download_folder, zip_filename)
    with open(dl_path, 'wb') as z_file:
        z_file.write(r.content)

unzip

In [5]:
os.chdir(download_folder)
for item in os.listdir(download_folder):
    if item.endswith('.zip'):
        file_name = os.path.abspath(item) # get full path of files
        zip_ref = zipfile.ZipFile(file_name) # create zipfile object
        zip_ref.extractall(download_folder) # extract file to dir
        zip_ref.close() # close file
        os.remove(file_name) # delete zipped file

Download the Bank Holding Company data in bulk

In [53]:
url = 'https://www.chicagofed.org/banking/financial-institution-reports/bhc-data'
root = 'https://www.chicagofed.org/'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
all_hrefs = soup.find_all('a')
all_links = [link.get('href') for link in all_hrefs]
csv_files = [dl for dl in all_links if dl is not None and '.csv' in dl]
csv_files = [dl.lower() for dl in csv_files]
csv_files = [dl[:dl.index('.csv')+len('.csv')] for dl in csv_files]
download_folder = 'C:/Users/kwang648/Downloads/banking/'

In [55]:
if not os.path.exists(download_folder):
    os.makedirs(download_folder)

for zip_file in zip_files:
    full_url = root + zip_file
    r = requests.get(full_url)
    zip_filename = os.path.basename(zip_file)
    dl_path = os.path.join(download_folder, zip_filename)
    with open(dl_path, 'wb') as z_file:
        z_file.write(r.content)

In [64]:
download_folder = 'C:/Users/kwang648/Downloads/banking/zipped'
os.chdir(download_folder)
for item in os.listdir(download_folder):
    file_name = os.path.abspath(item) # get full path of files
    zip_ref = zipfile.ZipFile(file_name) # create zipfile object
    zip_ref.extractall(download_folder) # extract file to dir
    zip_ref.close() # close file
    os.remove(file_name) # delete zipped file

convert xpt and text to csv

In [ ]:
os.chdir('C:/Users/kwang648/Downloads/banking data/Commercial Bank Data - 1976-2010')
for item in os.listdir('C:/Users/kwang648/Downloads/banking data/Commercial Bank Data - 1976-2010'):
    if item.endswith('.xpt'):
        os.system('python -m xport '+item+' > '+item[:item.index('.xpt')]+'.csv')
    elif item.endswith('.XPT'):
        os.system('python -m xport '+item+' > '+item[:item.index('.XPT')]+'.csv')

Convert text to csv, and merge

In [18]:
os.chdir('C:/Users/kwang648/Downloads/banking data/Commercial Bank Data - 2011-2023')
for item in os.listdir('C:/Users/kwang648/Downloads/banking data/Commercial Bank Data - 2011-2023'):
    new = pd.read_csv(item, delimiter = "\t")
    break

C:\tools\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252) have mixed types.Specify dt

In [22]:
new

,Reporting Period End Date,IDRSSD,FDIC Certificate Number,OCC Charter Number,OTS Docket Number,Primary ABA Routing Number,Financial Institution Name,Financial Institution Address,Financial Institution City,Financial Institution State,...,RCON5380,RCON5381,RCON5382,RCON5389,RCON5390,RCON5391,RCON5398,RCON5399,RCON5400,Unnamed: 253
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,LNS TO FOREIGN BKS-PAST DU 30-89 DYS,LNS TO FOREIGN BKS-PAST DU 90 DYS OR,LNS TO FOREIGN BKS-NONACCRUAL,"LNS TO FGN GOV,ETC.-PAS DU 30-89 DYS","LNS TO FGN GOV,ETC.-PAS DU 90 DYS OR","LNS TO FGN GOV,ETC.-NONACCRUAL","RVLVG,OPEN-END LNS-PAST DU 30-89 DYS","RVLVG,OPEN-END LNS-PAST DU 90 DYS OR","RVLVG,OPEN-END LNS-NONACCRUAL",NaN
1,2011-12-31,37.0,10057.0,0.0,16553.0,61107146.0,BANK OF HANCOCK COUNTY,12855 BROAD STREET,SPARTA,GA,...,NaN,NaN,NaN,0,0,0,0,0,0,NaN
2,2011-12-31,242.0,3850.0,0.0,0.0,81220537.0,FIRST COMMUNITY BANK XENIA-FLORA,260 FRONT STREET,XENIA,IL,...,NaN,NaN,NaN,0,0,0,0,0,0,NaN
3,2011-12-31,279.0,28868.0,0.0,2523.0,311972526.0,"MINEOLA COMMUNITY BANK, SSB",215 W BROAD,MINEOLA,TX,...,NaN,NaN,NaN,0,0,0,0,0,0,NaN
4,2011-12-31,354.0,14083.0,0.0,0.0,101107475.0,BISON STATE BANK,223 MAIN STREET,BISON,KS,...,NaN,NaN,NaN,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27404,2011-03-31,4165907.0,58961.0,0.0,0.0,65205549.0,LAKESIDE BANK,4735 NELSON ROAD,LAKE CHARLES,LA,...,NaN,NaN,NaN,0.0,0.0,0.0,0,0,0,NaN
27405,2011-03-31,4184186.0,59051.0,0.0,0.0,71004226.0,URBAN PARTNERSHIP BANK,7054 SOUTH JEFFERY AVENUE,CHICAGO,IL,...,0.0,0.0,0.0,0.0,0.0,0.0,322,664,0,NaN
27406,2011-03-31,4203911.0,59015.0,24965.0,0.0,101002949.0,"HILLCREST BANK, NATIONAL ASSOCIATION",11111 WEST 95TH STREET,OVERLAND PARK,KS,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,NaN
27407,2011-03-31,4210227.0,59052.0,24994.0,0.0,101006699.0,"BANK MIDWEST, NATIONAL ASSOCIATION","1111 MAIN STREET, SUITE 1600",KANSAS CITY,MO,...,0.0,0.0,0.0,0.0,0.0,0.0,335,0,34,NaN


In [20]:
test = pd.read_csv('C:/Users/kwang648/Downloads/banking data/test/call2011.csv')

In [5]:
def type_conversion():
    for column in new.columns:
        if infer_dtype(new[column])=='mixed-integer':
            new[column] = pd.to_numeric(new[column], errors='coerce')

In [8]:
os.chdir('C:/Users/kwang648/Downloads/banking data\call_csv - 2011-2023')
call_report = pd.DataFrame()
progress = 0
for item in os.listdir('C:/Users/kwang648/Downloads/banking data\call_csv - 2011-2023'):
    progress+=1
    print(progress)
    new = pd.read_csv(item,encoding = 'latin-1')
    type_conversion()
    call_report = pd.concat([call_report,new])

1
2
3
4
5
6
7


C:\tools\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (74,228) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


8


C:\tools\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (75) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


9


C:\tools\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (72) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


10
11


C:\tools\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (72,220) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


12
13


In [29]:
call_report

,Reporting_Period_End_Date,FDIC_Certificate_Number,OCC_Charter_Number,OTS_Docket_Number,Primary_ABA_Routing_Number,Financial_Institution_Name,Financial_Institution_Address,Financial_Institution_City,Financial_Institution_State,Financial_Institution_Zip_Code,...,RIADH034,RIADH035,RIADHK03,RIADHK04,RIADHT69,RIADHT70,RIADHT73,RIADHT74,RIADJ321,RIADJJ33
0,2011-12-31,10057,0,16553,61107146,BANK OF HANCOCK COUNTY,12855 BROAD STREET,SPARTA,GA,31087,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011-12-31,3850,0,0,81220537,FIRST COMMUNITY BANK XENIA-FLORA,260 FRONT STREET,XENIA,IL,62899,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2011-12-31,28868,0,2523,311972526,"MINEOLA COMMUNITY BANK, SSB",215 W BROAD,MINEOLA,TX,75773,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2011-12-31,14083,0,0,101107475,BISON STATE BANK,223 MAIN STREET,BISON,KS,67520,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011-12-31,10202,0,0,91208332,LOWRY STATE BANK,400 FLORENCE AVE.,LOWRY,MN,56349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4719,2023-03-31,59312,0,0,21315169,GS&L MUNICIPAL BANK,42 CHURCH STREET,GOUVERNEUR,NY,13642,...,NaN,NaN,1.0,1.0,262.0,0.0,0.0,0.0,NaN,0.0
4720,2023-03-31,59289,0,0,11402121,WALDEN MUTUAL BANK,66 N MAIN STREET,CONCORD,NH,3301,...,NaN,NaN,3.0,11.0,-689.0,0.0,0.0,0.0,NaN,76.0
4721,2023-03-31,59296,0,0,122245387,BANK IRVINE,"6201 OAK CANYON, SUITE 150",IRVINE,CA,92618,...,NaN,NaN,2.0,58.0,-797.0,0.0,0.0,0.0,NaN,212.0
4722,2023-03-31,57288,25281,0,122106497,"WESTERN ALLIANCE TRUST COMPANY, NATIONAL ASSOC...",1 EAST WASHINGTON STREET SUITE 1400,PHOENIX,AZ,85004,...,NaN,NaN,0.0,0.0,-3323.0,0.0,0.0,0.0,NaN,0.0
